In [2]:
from DataReader import DataReaderChallangePolimi2017
import implicit
from CB_asy import CB_asymmetric_cosine
from CF_IB_asy import CF_IB_asymmetric_cosine
from CF_UB_asy import CF_UB_asymmetric_cosine
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,TFIDFRecommender)
from sklearn.preprocessing import normalize
import numpy as np
import scipy.sparse as sp
import pickle

evaluation = True
rebuild = False
dr = DataReaderChallangePolimi2017(evaluation=evaluation,rebuild=rebuild)

urm = dr.getURM_csr()
t_p = dr.target_playlists
t_t = dr.target_tracks
if evaluation:
    filesslim = "./DumpData/sim_bpr_10e_100k2.npz" # or "./DumpData/sim_bpr_10e_100k.npz"
else:
    filesslim = "./DumpData/sim_bpr_10e_100k_fulltrain.npz"

Date Reader Challange Polimi 2017
Loading data from ./DumpData/DataReaderTrainTestDump
Dataset train-test loaded


In [3]:
#titles
ucm_titles = dr.getTitlesUCM_csr()
ucm = normalize(ucm_titles,axis=0)
model = BM25Recommender(K=100)
model.fit(ucm)
s = model.similarity
r_t = s.T*urm
r_t = normalize(r_t, axis=1)
if evaluation: print dr.evaluateMAP(r_t,verbose=False)
r_t = dr.reduceRM(r_t)

0.0108685277778


In [12]:
#owners
ucm_owners = dr.getOwnersUCM_csr()
ucm = normalize(ucm_owners,axis=0)
model = BM25Recommender(K=110)
model.fit(ucm)
s = model.similarity
r_o = s.T*urm
r_o = normalize(r_o, axis=0)
if evaluation: print dr.evaluateMAP(r_o,verbose=False)
r_o = dr.reduceRM(r_o)

0.0225329166667


In [ ]:
########## tuning ###########

In [7]:
# owners
def cb_owners(urm,ucm,k):
    ucm = normalize(ucm,axis=0)
    model = BM25Recommender(K=k)
    model.fit(ucm)
    s = model.similarity
    r_al = s.T*urm
    r_al = normalize(r_al, axis=0)
    return dr.evaluateMAP(r_al,verbose=False)

In [ ]:
#tune owners
urm = dr.getURM_csr()
ucm = dr.getOwnersUCM_csr()

for i in range(100,120,1):
    score= cb_owners(urm=urm,ucm=ucm,k=i)
    print ("k = {:3d} \t\tmap = {:1.5f}").format(i,score)